In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import os


In [2]:

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Definir el Generador
class Generator(nn.Module):
    def __init__(self, nz, ngf, nc):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            nn.ConvTranspose2d(nz, ngf*8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf*8),
            nn.ReLU(True),
            nn.ConvTranspose2d(ngf*8, ngf*4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf*4),
            nn.ReLU(True),
            nn.ConvTranspose2d(ngf*4, ngf*2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf*2),
            nn.ReLU(True),
            nn.ConvTranspose2d(ngf*2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            nn.ConvTranspose2d(ngf, nc, 4, 2, 1, bias=False),
            nn.Tanh()
        )

    def forward(self, input):
        return self.main(input)

# Definir el Discriminador
class Discriminator(nn.Module):
    def __init__(self, nc, ndf):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf, ndf*2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf*2),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf*2, ndf*4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf*4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf*4, ndf*8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf*8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf*8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        return self.main(input)

# Función para inicializar los pesos de la red
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)


In [3]:

# Configuración de parámetros
nz = 100    # Tamaño del vector de ruido de entrada al generador
ngf = 64    # Número de filtros en la última capa de desconvolución del generador
ndf = 64    # Número de filtros en la primera capa de convolución del discriminador
nc = 3      # Número de canales de la imagen (RGB)
lr = 0.0002 # Tasa de aprendizaje
beta1 = 0.5 # Beta1 para el optimizador Adam

# Crear modelos
netG = Generator(nz, ngf, nc).to(device)
netD = Discriminator(nc, ndf).to(device)

# Inicializar pesos
netG.apply(weights_init)
netD.apply(weights_init)

# Configurar optimizadores y criterio de pérdida
criterion = nn.BCELoss()
optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))


In [4]:

# Función para entrenar el GAN y generar imágenes
def train_gan(num_epochs, dataloader, device, netG, netD, optimizerG, optimizerD):
    for epoch in range(num_epochs):
        for i, data in enumerate(dataloader, 0):
            
            # (1) Actualizar el Discriminador: maximizar log(D(x)) + log(1 - D(G(z)))
            
            netD.zero_grad()
            real_cpu = data[0].to(device)
            b_size = real_cpu.size(0)
            label = torch.full((b_size,), 1, dtype=torch.float, device=device)
            output = netD(real_cpu).view(-1)
            errD_real = criterion(output, label)
            errD_real.backward()
            D_x = output.mean().item()

            noise = torch.randn(b_size, nz, 1, 1, device=device)
            fake = netG(noise)
            label.fill_(0)
            output = netD(fake.detach()).view(-1)
            errD_fake = criterion(output, label)
            errD_fake.backward()
            D_G_z1 = output.mean().item()
            errD = errD_real + errD_fake
            optimizerD.step()

            
            # (2) Actualizar el Generador: maximizar log(D(G(z)))
            
            netG.zero_grad()
            label.fill_(1)
            output = netD(fake).view(-1)
            errG = criterion(output, label)
            errG.backward()
            D_G_z2 = output.mean().item()
            optimizerG.step()

            # Imprimir estadísticas del entrenamiento
            if i % 50 == 0:
                print(f'Epoch [{epoch}/{num_epochs}], Batch [{i}/{len(dataloader)}], '
                      f'Loss_D: {errD.item():.4f}, Loss_G: {errG.item():.4f}, '
                      f'D(x): {D_x:.4f}, D(G(z)): {D_G_z1:.4f} / {D_G_z2:.4f}')

        # # Guardar imágenes generadas al final de cada época
        # with torch.no_grad():
        #     fake = netG(torch.randn(64, nz, 1, 1, device=device))
        #     torchvision.utils.save_image(fake, f'generated_images/fake_samples_epoch_{epoch+1:03d}.png', normalize=True)

# Dataset de imágenes de melanomas
dataset_path = './train'  # Ruta al dataset de melanomas

# Transformaciones para el dataset
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])


: 

In [5]:

# Crear DataLoader para el dataset de entrenamiento
batch_size = 32
dataset = torchvision.datasets.ImageFolder(root=dataset_path, transform=transform)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=2)

# Entrenar el GAN y guardar imágenes generadas
num_epochs = 10
train_gan(num_epochs, dataloader, device, netG, netD, optimizerG, optimizerD)

print('Entrenamiento del GAN completado')


Epoch [0/10], Batch [0/372], Loss_D: 1.7240, Loss_G: 5.0149, D(x): 0.4569, D(G(z)): 0.4290 / 0.0073
Epoch [0/10], Batch [50/372], Loss_D: 0.2065, Loss_G: 7.6785, D(x): 0.8850, D(G(z)): 0.0046 / 0.0009
Epoch [0/10], Batch [100/372], Loss_D: 0.0061, Loss_G: 37.0601, D(x): 0.9942, D(G(z)): 0.0000 / 0.0000
Epoch [0/10], Batch [150/372], Loss_D: 0.0000, Loss_G: 35.9701, D(x): 1.0000, D(G(z)): 0.0000 / 0.0000
Epoch [0/10], Batch [200/372], Loss_D: 0.0080, Loss_G: 33.7134, D(x): 0.9929, D(G(z)): 0.0000 / 0.0000
Epoch [0/10], Batch [250/372], Loss_D: 1.4964, Loss_G: 32.6844, D(x): 0.5727, D(G(z)): 0.0000 / 0.0000
Epoch [0/10], Batch [300/372], Loss_D: 0.1959, Loss_G: 5.1649, D(x): 0.9375, D(G(z)): 0.0799 / 0.0075
Epoch [0/10], Batch [350/372], Loss_D: 0.1479, Loss_G: 5.8011, D(x): 0.9703, D(G(z)): 0.0945 / 0.0063
Epoch [1/10], Batch [0/372], Loss_D: 0.5801, Loss_G: 3.2436, D(x): 0.7680, D(G(z)): 0.1455 / 0.0830
Epoch [1/10], Batch [50/372], Loss_D: 0.4727, Loss_G: 3.9321, D(x): 0.8507, D(G(z))